In [4]:

import pandas as pd
import numpy as np
import math
import seaborn as sns

# Modeling preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector as selector
from sklearn.pipeline import Pipeline

# Modeling and resampling
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedKFold

Reading the dataset

http://archive.ics.uci.edu/ml/datasets/Bank+Marketing

In [5]:
bank = pd.read_csv("data/bank-additional-full.csv", sep= ";")


EDA

In [6]:
bank.sample(10)


,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
37508,35,admin.,single,high.school,no,no,yes,cellular,aug,thu,...,1,999,1,failure,-2.9,92.201,-31.4,0.873,5076.2,no
19059,40,technician,married,professional.course,no,no,no,cellular,aug,tue,...,1,999,0,nonexistent,1.4,93.444,-36.1,4.968,5228.1,no
40260,61,admin.,married,unknown,no,yes,yes,cellular,jul,mon,...,3,999,1,failure,-1.7,94.215,-40.3,0.889,4991.6,no
4359,27,admin.,married,high.school,unknown,no,yes,telephone,may,tue,...,3,999,0,nonexistent,1.1,93.994,-36.4,4.856,5191.0,no
35134,35,admin.,married,high.school,unknown,yes,no,cellular,may,fri,...,2,999,1,failure,-1.8,92.893,-46.2,1.250,5099.1,no
8339,43,unemployed,married,basic.9y,no,no,no,telephone,jun,tue,...,28,999,0,nonexistent,1.4,94.465,-41.8,4.864,5228.1,no
33179,35,blue-collar,married,high.school,no,yes,no,cellular,may,tue,...,1,999,0,nonexistent,-1.8,92.893,-46.2,1.291,5099.1,no
1004,47,admin.,married,university.degree,no,unknown,unknown,telephone,may,wed,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.856,5191.0,no
29291,31,entrepreneur,married,university.degree,unknown,yes,no,cellular,apr,fri,...,2,999,1,failure,-1.8,93.075,-47.1,1.405,5099.1,no
16117,32,blue-collar,married,basic.4y,no,yes,no,cellular,jul,tue,...,1,999,0,nonexistent,1.4,93.918,-42.7,4.961,5228.1,no


In [7]:
bank.shape

(41188, 21)

we are going to remove "duration" because this attribute highly affects the output target (e.g., if duration=0 then y='no'). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic

In [17]:
del bank["duration"]

In [18]:
bank.shape

(41188, 20)

In [8]:
bank.columns

Index(['age', 'job', 'marital', 'education', 'default', 'housing', 'loan',
       'contact', 'month', 'day_of_week', 'duration', 'campaign', 'pdays',
       'previous', 'poutcome', 'emp.var.rate', 'cons.price.idx',
       'cons.conf.idx', 'euribor3m', 'nr.employed', 'y'],
      dtype='object')

##2
Assess the dataset for missingness.


In [14]:
bank.isnull().sum()

age               0
job               0
marital           0
education         0
default           0
housing           0
loan              0
contact           0
month             0
day_of_week       0
duration          0
campaign          0
pdays             0
previous          0
poutcome          0
emp.var.rate      0
cons.price.idx    0
cons.conf.idx     0
euribor3m         0
nr.employed       0
y                 0
dtype: int64

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,73,retired,married,professional.course,no,yes,no,cellular,nov,fri,...,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,yes
41184,46,blue-collar,married,professional.course,no,no,no,cellular,nov,fri,...,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,no
41185,56,retired,married,university.degree,no,yes,no,cellular,nov,fri,...,2,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,no
41186,44,technician,married,professional.course,no,no,no,cellular,nov,fri,...,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,yes


split the dataset into traing and testing set

In [ ]:
y = bank["y"]
train, test = train_test_split(bank, test_size=0.3, 
random_state=123, stratify=y)

In [ ]:
f"raw data dimensions: {bank.shape}; training dimensions: {train.shape}; testing dimensions:  {test.shape}"

 Extract features and response


In [ ]:
features = train.drop(columns="y")
label = train["y"]

In [ ]:
results = grid_search.fit(features, label)

In [ ]:
categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])
preprocessor = ColumnTransformer(transformers=[('cat', categorical_transformer, selector(dtype_include="object"))])

knn_fit = Pipeline(steps=[('preprocessor', preprocessor),
                          ('knn', KNeighborsRegressor(metric='euclidean'))])

Specify resampling strategy

In [ ]:
cv = RepeatedKFold(n_splits=10, n_repeats=5)

Create grid of hyperparameter values

In [ ]:
hyper_grid = {'knn__n_neighbors': range(3, 26)}

Tune a knn model using grid search

In [ ]:
grid_search = GridSearchCV(knn_fit, hyper_grid, cv=cv, scoring='neg_mean_squared_error')